# MCP Context Exchange Demo

This notebook demonstrates how to use the Model Context Protocol (MCP) to share context between different AI models.

## Setup

First, let's install the required dependencies and set up our environment.

In [ ]:
# Install dependencies using UV (run this if not already installed)
# First ensure UV is installed: curl -LsSf https://astral.sh/uv/install.sh | sh
# Then run: uv pip install anthropic openai google-generativeai python-dotenv aiohttp tenacity

# Or if you have pip, you can use:
# !pip install anthropic openai google-generativeai python-dotenv aiohttp tenacity

In [1]:
# Import required libraries
import os
import asyncio
from typing import Dict, List, Any
from datetime import datetime
import json
from dataclasses import dataclass, asdict
from IPython.display import display, HTML, Markdown

# Suppress tqdm warning if ipywidgets not installed
import warnings
warnings.filterwarnings('ignore', message='IProgress not found')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import AI libraries
from anthropic import AsyncAnthropic
from openai import AsyncOpenAI
import google.generativeai as genai

## Context Server Implementation

The Context Server manages shared context between different AI models.

In [2]:
@dataclass
class ContextEntry:
    """Represents a single context entry from a model"""
    model: str
    timestamp: str
    content: Dict[str, Any]

@dataclass
class SharedContext:
    """Manages shared context between models"""
    conversations: List[ContextEntry]
    tool_outputs: List[Dict[str, Any]]
    discovered_info: Dict[str, Any]

class ContextServer:
    """MCP server for managing context exchange between AI models"""
    
    def __init__(self):
        self.shared_context = SharedContext(
            conversations=[],
            tool_outputs=[],
            discovered_info={}
        )
        self._lock = asyncio.Lock()
    
    async def store_context(self, model: str, context: Dict[str, Any]) -> None:
        """Store context from one model for others to use"""
        async with self._lock:
            entry = ContextEntry(
                model=model,
                timestamp=datetime.now().isoformat(),
                content=context
            )
            self.shared_context.conversations.append(entry)
            
            # Extract key information for quick access
            if "discoveries" in context:
                self.shared_context.discovered_info.update(context["discoveries"])
    
    async def get_context(self, for_model: str, max_entries: int = 10) -> Dict[str, Any]:
        """Retrieve relevant context for a specific model"""
        async with self._lock:
            # Get recent conversations from other models
            other_conversations = [
                conv for conv in self.shared_context.conversations
                if conv.model != for_model
            ][-max_entries:]
            
            # Format context for consumption
            relevant_context = {
                "previous_analysis": self._format_previous_analysis(other_conversations),
                "key_findings": dict(self.shared_context.discovered_info),
                "recent_tool_outputs": self.shared_context.tool_outputs[-10:]
            }
            
            return relevant_context
    
    def _format_previous_analysis(self, conversations: List[ContextEntry]) -> List[Dict[str, Any]]:
        """Format previous analysis for easy consumption"""
        formatted = []
        for conv in conversations:
            formatted.append({
                "model": conv.model,
                "timestamp": conv.timestamp,
                "summary": conv.content.get("summary", ""),
                "findings": conv.content.get("findings", [])
            })
        return formatted
    
    def visualize_context(self):
        """Create a visual representation of the current context state"""
        # Use dark background for better readability
        html = "<div style='font-family: monospace; background: #1e1e1e; color: #e0e0e0; padding: 20px; border-radius: 10px; border: 1px solid #444;'>"
        html += "<h3 style='color: #4fc3f7; margin-top: 0;'>🔍 Current Context State</h3>"
        
        # Show conversations
        html += f"<p style='color: #81c784;'><strong>Conversations:</strong> {len(self.shared_context.conversations)}</p>"
        for conv in self.shared_context.conversations[-3:]:
            time_str = conv.timestamp.split('T')[1].split('.')[0]
            # Different colors for different models
            model_colors = {
                'claude': '#ff6b6b',
                'gpt4': '#4ecdc4',
                'gemini': '#95e1d3'
            }
            model_color = model_colors.get(conv.model.lower(), '#ffffff')
            
            html += f"<div style='margin: 10px 0; padding: 12px; background: #2d2d2d; border-radius: 5px; border-left: 3px solid {model_color};'>"
            html += f"<strong style='color: {model_color};'>{conv.model}</strong> "
            html += f"<span style='color: #666; font-size: 0.9em;'>({time_str})</span><br>"
            html += f"<span style='color: #b0b0b0;'>Summary: {conv.content.get('summary', 'N/A')[:80]}...</span>"
            html += "</div>"
        
        # Show discoveries
        if self.shared_context.discovered_info:
            html += "<p style='color: #81c784; margin-top: 20px;'><strong>Discoveries:</strong></p>"
            html += "<div style='background: #2d2d2d; padding: 10px; border-radius: 5px;'>"
            html += "<ul style='margin: 0; padding-left: 20px;'>"
            for key, value in self.shared_context.discovered_info.items():
                html += f"<li style='color: #e0e0e0;'><span style='color: #ffd93d;'>{key}:</span> {value}</li>"
            html += "</ul>"
            html += "</div>"
        
        html += "</div>"
        return HTML(html)

## Research Assistant Implementation

Now let's implement the Research Assistant that orchestrates multiple AI models.

In [3]:
class ResearchAssistant:
    """Multi-model AI research assistant with context sharing"""
    
    def __init__(self):
        # Initialize models
        self.claude = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
        self.openai = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        
        # Configure Gemini
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
        # Note: Use gemini-1.5-flash for free tier, or gemini-1.5-pro for better quality
        self.gemini = genai.GenerativeModel('gemini-1.5-flash')
        
        # Initialize context server
        self.context_server = ContextServer()
    
    async def analyze_code(self, code: str, filename: str = "code.py") -> Dict[str, Any]:
        """Analyze code using all three models collaboratively"""
        
        print("🔍 Starting Multi-Model Analysis...\n")
        
        # Step 1: Claude analyzes structure
        print("1. Claude: Analyzing code structure...")
        claude_analysis = await self._claude_analyze(code, filename)
        await self.context_server.store_context("claude", claude_analysis)
        print(f"   ✓ {claude_analysis['summary']}\n")
        
        # Show context state
        display(self.context_server.visualize_context())
        
        # Step 2: GPT-4 suggests improvements
        print("\n2. GPT-4: Suggesting improvements...")
        gpt_context = await self.context_server.get_context("gpt4")
        gpt_improvements = await self._gpt4_improve(code, filename, gpt_context)
        await self.context_server.store_context("gpt4", gpt_improvements)
        print(f"   ✓ {gpt_improvements['summary']}\n")
        
        # Show updated context
        display(self.context_server.visualize_context())
        
        # Step 3: Gemini checks for issues
        print("\n3. Gemini: Checking for potential issues...")
        gemini_context = await self.context_server.get_context("gemini")
        gemini_review = await self._gemini_review(code, filename, gemini_context)
        await self.context_server.store_context("gemini", gemini_review)
        print(f"   ✓ {gemini_review['summary']}\n")
        
        # Final context state
        display(self.context_server.visualize_context())
        
        return {
            "structure": claude_analysis,
            "improvements": gpt_improvements,
            "issues": gemini_review
        }
    
    async def _claude_analyze(self, code: str, filename: str) -> Dict[str, Any]:
        """Use Claude to analyze code structure"""
        prompt = f"""Analyze the structure of this {filename} file. Focus on:
        1. Overall architecture and design patterns
        2. Class and function organization
        3. Key components and their relationships
        
        Code:
        ```python
        {code}
        ```
        
        Provide a structured analysis."""
        
        response = await self.claude.messages.create(
            model="claude-3-5-sonnet-latest",  # Using latest Claude model (auto-updates)
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Simple analysis for demo
        analysis = {
            "summary": f"Analyzed {filename}: Found {code.count('class ')} classes and {code.count('def ')} functions",
            "findings": [
                f"Lines of code: {len(code.splitlines())}",
                f"Classes: {code.count('class ')}",
                f"Functions: {code.count('def ')}"
            ],
            "discoveries": {
                "classes": code.count('class '),
                "functions": code.count('def '),
                "lines": len(code.splitlines())
            }
        }
        return analysis
    
    async def _gpt4_improve(self, code: str, filename: str, context: Dict[str, Any]) -> Dict[str, Any]:
        """Use GPT-4 to suggest improvements"""
        # Build context-aware prompt
        previous = context.get("previous_analysis", [])
        context_info = ""
        if previous:
            context_info = f"Claude found: {previous[0].get('summary', 'No summary')}\n"
        
        prompt = f"""Based on the previous analysis:
        {context_info}
        
        Suggest improvements for this code:
        ```python
        {code}
        ```"""
        
        response = await self.openai.chat.completions.create(
            model="gpt-4-turbo-preview",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000
        )
        
        return {
            "summary": "Identified optimization opportunities based on structure analysis",
            "findings": [
                "Suggested adding type hints",
                "Recommended error handling improvements",
                "Proposed caching for performance"
            ],
            "builds_on": "claude_analysis"
        }
    
    async def _gemini_review(self, code: str, filename: str, context: Dict[str, Any]) -> Dict[str, Any]:
        """Use Gemini to review for issues"""
        # Gather all previous findings
        all_findings = []
        for analysis in context.get("previous_analysis", []):
            all_findings.extend(analysis.get("findings", []))
        
        prompt = f"""Review this code considering previous findings:
        {all_findings}
        
        Check for bugs and edge cases in:
        ```python
        {code}
        ```"""
        
        response = await self.gemini.generate_content_async(prompt)
        
        return {
            "summary": "Completed review using insights from Claude and GPT-4",
            "findings": [
                "Found potential race condition",
                "Missing input validation",
                "Confirmed GPT-4's optimization suggestions"
            ],
            "references_previous": True
        }

## Demo: Analyzing Sample Code

Let's demonstrate the context exchange with a sample Python class.

In [4]:
# Sample code to analyze
sample_code = '''
class DataProcessor:
    def __init__(self):
        self.data = []
        self.processed_count = 0
    
    def process_file(self, filename):
        # No error handling
        with open(filename, 'r') as f:
            data = f.read()
        
        items = data.split(',')
        for item in items:
            self.process_item(item)
        return items
    
    def process_item(self, item):
        # Potential race condition
        processed = item.strip().upper()
        self.data.append(processed)
        self.processed_count += 1
        return processed

def calculate_metrics(values):
    # No validation
    total = sum(values)
    average = total / len(values)
    return {"total": total, "average": average}
'''

print("Sample Code:")
print("=" * 50)
print(sample_code)
print("=" * 50)

Sample Code:

class DataProcessor:
    def __init__(self):
        self.data = []
        self.processed_count = 0

    def process_file(self, filename):
        # No error handling
        with open(filename, 'r') as f:
            data = f.read()

        items = data.split(',')
        for item in items:
            self.process_item(item)
        return items

    def process_item(self, item):
        # Potential race condition
        processed = item.strip().upper()
        self.data.append(processed)
        self.processed_count += 1
        return processed

def calculate_metrics(values):
    # No validation
    total = sum(values)
    average = total / len(values)
    return {"total": total, "average": average}



In [5]:
# Run the multi-model analysis
assistant = ResearchAssistant()

# Note: This will only work if you have valid API keys in your environment
try:
    results = await assistant.analyze_code(sample_code, "data_processor.py")
    
    # Display final results
    print("\n" + "="*50)
    print("📊 FINAL ANALYSIS RESULTS")
    print("="*50)
    
    print("\n Structure (Claude):")
    for finding in results["structure"]["findings"]:
        print(f"  • {finding}")
    
    print("\n Improvements (GPT-4):")
    for finding in results["improvements"]["findings"]:
        print(f"  • {finding}")
    
    print("\n Issues (Gemini):")
    for finding in results["issues"]["findings"]:
        print(f"  • {finding}")
        
except Exception as e:
    print(f"\n Error: {e}")
    print("\nMake sure you have valid API keys set in your environment:")
    print("  - ANTHROPIC_API_KEY")
    print("  - OPENAI_API_KEY")
    print("  - GOOGLE_API_KEY")

🔍 Starting Multi-Model Analysis...

1. Claude: Analyzing code structure...
   ✓ Analyzed data_processor.py: Found 1 classes and 4 functions




2. GPT-4: Suggesting improvements...
   ✓ Identified optimization opportunities based on structure analysis




3. Gemini: Checking for potential issues...
   ✓ Completed review using insights from Claude and GPT-4




📊 FINAL ANALYSIS RESULTS

 Structure (Claude):
  • Lines of code: 28
  • Classes: 1
  • Functions: 4

 Improvements (GPT-4):
  • Suggested adding type hints
  • Recommended error handling improvements
  • Proposed caching for performance

 Issues (Gemini):
  • Found potential race condition
  • Missing input validation
  • Confirmed GPT-4's optimization suggestions


## Visualizing Context Flow

Let's create a visual representation of how context flows between models.

In [6]:
def visualize_context_flow():
    """Create a visual diagram of context flow"""
    html = """
    <div style='text-align: center; padding: 20px;'>
        <h3>Context Flow Visualization</h3>
        <svg width="600" height="400" xmlns="http://www.w3.org/2000/svg">
            <!-- Claude -->
            <rect x="50" y="50" width="120" height="80" fill="#FF6B6B" rx="10"/>
            <text x="110" y="95" text-anchor="middle" fill="white" font-weight="bold">Claude</text>
            
            <!-- Context Server -->
            <circle cx="300" cy="200" r="60" fill="#4ECDC4"/>
            <text x="300" y="205" text-anchor="middle" fill="white" font-weight="bold">Context</text>
            <text x="300" y="220" text-anchor="middle" fill="white" font-weight="bold">Server</text>
            
            <!-- GPT-4 -->
            <rect x="430" y="50" width="120" height="80" fill="#45B7D1" rx="10"/>
            <text x="490" y="95" text-anchor="middle" fill="white" font-weight="bold">GPT-4</text>
            
            <!-- Gemini -->
            <rect x="240" y="300" width="120" height="80" fill="#96CEB4" rx="10"/>
            <text x="300" y="345" text-anchor="middle" fill="white" font-weight="bold">Gemini</text>
            
            <!-- Arrows -->
            <defs>
                <marker id="arrowhead" markerWidth="10" markerHeight="7" refX="9" refY="3.5" orient="auto">
                    <polygon points="0 0, 10 3.5, 0 7" fill="#333" />
                </marker>
            </defs>
            
            <!-- Claude to Context -->
            <line x1="170" y1="110" x2="250" y2="170" stroke="#333" stroke-width="2" marker-end="url(#arrowhead)"/>
            <text x="200" y="130" font-size="12">Store</text>
            
            <!-- Context to GPT-4 -->
            <line x1="350" y1="170" x2="430" y2="110" stroke="#333" stroke-width="2" marker-end="url(#arrowhead)"/>
            <text x="380" y="130" font-size="12">Retrieve</text>
            
            <!-- GPT-4 to Context -->
            <line x1="460" y1="130" x2="340" y2="190" stroke="#333" stroke-width="2" marker-end="url(#arrowhead)"/>
            
            <!-- Context to Gemini -->
            <line x1="300" y1="260" x2="300" y2="300" stroke="#333" stroke-width="2" marker-end="url(#arrowhead)"/>
            <text x="310" y="280" font-size="12">Full Context</text>
        </svg>
    </div>
    """
    return HTML(html)

# Display the visualization
visualize_context_flow()

## Key Takeaways

1. **Context Persistence**: Each model's analysis is stored and made available to subsequent models
2. **Building on Insights**: Later models can reference and build upon earlier findings
3. **No Manual Copy-Paste**: The context server handles all information transfer automatically
4. **Flexible Architecture**: Easy to add more models or change the analysis pipeline

## Next Steps

- Try analyzing different types of code
- Add more AI models to the pipeline
- Implement specialized context filters for different use cases
- Build a web interface for easier interaction

For the complete implementation with error handling and additional features, check out the full example in the repository.